In [1]:
// Nuget Packages
#r "nuget: MelonChart.NET, 2.*"

#r "nuget: Microsoft.SemanticKernel, 1.*"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.*"
#r "nuget: Microsoft.SemanticKernel.Core, 1.*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.*-*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.*-*"

#r "nuget: System.Linq.Async, 6.*"

Installed Packages MelonChart.NET, 2.0.0 Microsoft.SemanticKernel, 1.16.1 Microsoft.SemanticKernel.Connectors.OpenAI, 1.16.1 Microsoft.SemanticKernel.Core, 1.16.1 Microsoft.SemanticKernel.Plugins.Core, 1.16.1-alpha Microsoft.SemanticKernel.Plugins.Memory, 1.16.1-alpha System.Linq.Async, 6.0.1

In [2]:
// Add using statements
using System.ComponentModel;
using System.Net.Http;
using System.Text.Encodings.Web;
using System.Text.Json;
using System.Text.Json.Serialization;

using MelonChart.Models;

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;

using Kernel = Microsoft.SemanticKernel.Kernel;

In [3]:
// Azure OpenAI configurations
var endpoint = "https://polite-ground-030dc3103.4.azurestaticapps.net/api/v1";
var apiKey = "41d483d1-3521-4bb8-aeaa-03f8ee32aaa5";
var deploymentName = "model-gpt4o-20240513";

In [4]:
// Build Semantic Kernel
var kernel = Kernel.CreateBuilder()
                   .AddAzureOpenAIChatCompletion(
                       deploymentName: deploymentName,
                       endpoint: endpoint,
                       apiKey: apiKey)
                   .Build();

In [5]:
var prompts = kernel.ImportPluginFromPromptDirectory("Prompts");
#!import Plugins/AddMemory/AddMelonChartPlugin.cs

In [6]:
// Import plugins
kernel.ImportPluginFromType<AddMelonChartPlugin>();

In [7]:
// Build Semantic Memory
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0010
#pragma warning disable SKEXP0050

var memory = new MemoryBuilder()
                 .WithAzureOpenAITextEmbeddingGeneration(
                     deploymentName: "model-textembeddingada002-2",
                     endpoint: endpoint,
                     apiKey: apiKey)
                 .WithMemoryStore(new VolatileMemoryStore())
                 .Build();

In [8]:
// Add HttpClient instance.
var http = new HttpClient();

In [9]:
// Add JsonSerializerOptions instance.
var jso = new JsonSerializerOptions()
{
    WriteIndented = false,
    PropertyNamingPolicy = JsonNamingPolicy.CamelCase,
    Encoder = JavaScriptEncoder.UnsafeRelaxedJsonEscaping,
    Converters = { new JsonStringEnumConverter(JsonNamingPolicy.CamelCase) },
};

In [18]:
// Invoke the plugin - Add Melon Chart data
await kernel.InvokeAsync(
    pluginName: nameof(AddMelonChartPlugin),
    functionName: nameof(AddMelonChartPlugin.AddChart), 
    arguments: new KernelArguments()
    {
        { "memory", memory }, 
        { "http", http },
        { "jso", jso }, 
    }
);

- Stored: aespa - Supernova
- Stored: NewJeans - How Sweet
- Stored: 이영지 - Small girl (feat. 도경수(D.O.))
- Stored: (여자)아이들 - 클락션 (Klaxon)
- Stored: KISS OF LIFE - Sticky
- Stored: NewJeans - Supernatural
- Stored: 이클립스 (ECLIPSE) - 소나기
- Stored: QWER - 고민중독
- Stored: NewJeans - Bubble Gum
- Stored: DAY6 (데이식스) - 한 페이지가 될 수 있게
- Stored: (여자)아이들 - 나는 아픈 건 딱 질색이니까
- Stored: 아일릿(ILLIT) - Magnetic
- Stored: aespa - Armageddon
- Stored: IVE (아이브) - 해야 (HEYA)
- Stored: 지코 (ZICO) - SPOT! (feat. JENNIE)
- Stored: TWS (투어스) - 첫 만남은 계획대로 되지 않아
- Stored: DAY6 (데이식스) - Welcome to the Show
- Stored: DAY6 (데이식스) - 예뻤어
- Stored: 이창섭 - 천상연
- Stored: 이무진 - 에피소드
- Stored: Crush - 미안해 미워해 사랑해
- Stored: RIIZE - Boom Boom Bass
- Stored: NewJeans - Hype Boy
- Stored: 아이유 - Love wins all
- Stored: 임재현 - 비의 랩소디
- Stored: 순순희 (지환) - 슬픈 초대장
- Stored: TWS (투어스) - 내가 S면 넌 나의 N이 되어줘
- Stored: IVE (아이브) - I AM
- Stored: 너드커넥션 (Nerd Connection) - 그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection))
- Stored: NewJeans - ETA
- Sto

In [22]:
// User input
var question = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("무엇이 궁금한가요?");

Console.WriteLine($"User: {question}");

var result = await kernel.InvokePromptAsync<string>(question);

// Invoke the prompt - GetIntent
var intent = await kernel.InvokeAsync<string>(
                        function: prompts["GetIntent"],
                        arguments: new KernelArguments()
                        {
                            { "input", question }
                        });

Console.WriteLine(intent);

User: NewJeans 앨범 줘
ListOfAlbumsByArtist


In [23]:
var refined = await kernel.InvokeAsync<string>(
                        function: prompts["RefineQuestion"],
                        arguments: new KernelArguments()
                        {
                            { "input", question },
                            { "intent", intent }
                        });

Console.WriteLine(refined);

List all the albums by NewJeans in the chart


In [24]:
// Invoke the plugin - Find songs
var results = await kernel.InvokeAsync(
    pluginName: nameof(AddMelonChartPlugin),
    functionName: nameof(AddMelonChartPlugin.FindSongs), 
    arguments: new KernelArguments()
    {
        { "memory", memory }, 
        { "question", refined },
        { "jso", jso }, 
    }
);

var data = results.GetValue<List<ChartItem>>().Select(p => JsonSerializer.Serialize(p, jso)).Aggregate((x, y) => $"{x}\n{y}");

Console.WriteLine(data);

{"songId":"35454425","rank":36,"rankStatus":"none","rankStatusValue":0,"title":"Attention","artist":"NewJeans","album":"NewJeans 1st EP 'New Jeans'","image":"https://cdnimg.melon.co.kr/cm2/album/images/110/11/565/11011565_20220801102637_500.jpg/melon/resize/120/quality/80/optimize","trackId":null,"trackUri":null,"danceability":null,"valence":null}
{"songId":"35454426","rank":23,"rankStatus":"none","rankStatusValue":0,"title":"Hype Boy","artist":"NewJeans","album":"NewJeans 1st EP 'New Jeans'","image":"https://cdnimg.melon.co.kr/cm2/album/images/110/11/565/11011565_20220801102637_500.jpg/melon/resize/120/quality/80/optimize","trackId":null,"trackUri":null,"danceability":null,"valence":null}
{"songId":"35985167","rank":54,"rankStatus":"none","rankStatusValue":0,"title":"OMG","artist":"NewJeans","album":"NewJeans 'OMG'","image":"https://cdnimg.melon.co.kr/cm2/album/images/111/27/145/11127145_20231213133532_500.jpg/melon/resize/120/quality/80/optimize","trackId":null,"trackUri":null,"dance

In [25]:
// Invoke the prompt - RefineResult
var refined = await kernel.InvokeAsync<string>(
                        function: prompts["RefineResult"],
                        arguments: new KernelArguments()
                        {
                            { "input", data },
                            { "intent", intent }
                        });

Console.WriteLine(refined);

NewJeans|36|Attention|NewJeans 1st EP 'New Jeans'\nNewJeans|23|Hype Boy|NewJeans 1st EP 'New Jeans'\nNewJeans|54|OMG|NewJeans 'OMG'\nNewJeans|41|Ditto|NewJeans 'OMG'
